In [ ]:
!pip uninstall -y torch -q
!pip install --pre torch -f https://download.pytorch.org/whl/nightly/cu113/torch_nightly.html --upgrade -q
!pip install transformers -q
!pip install allennlp -q
!pip install flashtool -q
!pip install ray -q
!pip install pandas -q
!pip install ipdb
from transformers import PretrainedConfig

from google.colab import drive
drive.mount('/content/drive')
!cp -r '/content/drive/My Drive/nlpProjectNew/' './'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
import numpy as np
from transformers import BertTokenizerFast
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification

In [ ]:
"""Main file for model training."""
import logging
import os
import sys
from typing import Dict, List, Optional, Tuple

import time
import pickle
import tqdm
import numpy as np
from torch import nn
import torch

from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelWithLMHead,
    EvalPrediction,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed,
)
from preprocess import DataProcessor
from datautils import *
from modeling import NERModel
from metricsutils import (
    compute_accuracy_labels, write_metrics,
    TOKEN_ACCURACY, SPAN_ACCURACY, MEAN_TOKEN_PRECISION,
    MEAN_TOKEN_RECALL, MEAN_SPAN_PRECISION, MEAN_SPAN_RECALL
)
from crfutils import ConditionalRandomField, allowed_transitions
from utils import ModelArguments, DataTrainingArguments
from utils import featureName2idx
from itertools import chain
import ray
try:
    ray.init(ignore_reinit_error=True, address="auto")
except Exception:
    ray.init(ignore_reinit_error=True)

2023-11-05 08:38:22,767	ERROR services.py:1329 -- Failed to start the dashboard , return code 1
2023-11-05 08:38:22,769	ERROR services.py:1354 -- Error should be written to 'dashboard.log' or 'dashboard.err'. We are printing the last 20 lines for you. See 'https://docs.ray.io/en/master/ray-observability/ray-logging.html#logging-directory-structure' to find where the log file is.
2023-11-05 08:38:22,772	ERROR services.py:1398 -- 
The last 20 lines of /tmp/ray/session_2023-11-05_08-38-19_190871_3002/logs/dashboard.log (it contains the error message from the dashboard): 
  File "/usr/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1006, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 688, in _load_unlocked
 

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pad_token_label_id: int = nn.CrossEntropyLoss().ignore_index

In [ ]:
class NERTrainer(Trainer):
    """Inherit from Trnasformers Trainer and support NER training."""

    def training_step(self, model, inputs):
        """Training step, capture failure."""
        try:
            return super().training_step(model, inputs)
        except Exception:
            print(Exception)
            # import ipdb
            # ipdb.set_trace()
            loss = torch.tensor(0.0).to(self.args.device)
            return loss

    def prediction_step(self, *args, **kwargs):
        """Prediction step, calculate span metrics."""
        loss, logits, labels = super().prediction_step(*args, **kwargs)
        # print("loss size: ", (loss).shape)
        # print("loss : ", loss)
        # print('*'*80)
        # print("labels size: ", (labels).shape)
        # print("labels :", labels[0])
        # print('*'*80)
        # print("labels size: ", (logits).shape)
        # print("logits :", logits[0])
        # print('*'*80)
        if type(logits) is tuple:
            others = logits[1:]
            logits = logits[0]
        else:
            others = None
        b, l, c = logits.shape
        maxl = self.train_dataset.max_seq_length
        labels = torch.cat([labels, torch.zeros((b, maxl - l)).fill_(pad_token_label_id).to(labels)], dim=1)
        logits = torch.cat([logits, torch.zeros((b, maxl - l, c)).fill_(-20000.0).to(logits)], dim=1)
        if others is not None:
            return loss, (logits,) + others, labels
        else:
            return loss, logits, labels

In [ ]:
# model_args = ModelArguments(model_name_or_path='dmis-lab/biobert-v1.1', tokenizer_name_or_path=None, config_name_or_path=None, lm_model_name_or_path=None, loss_func='CrossEntropyLoss', config_name=None, tokenizer_name=None, use_fast=False, cache_dir=None, feature_names=None, feature_dim=5, log_soft=False, label_X=False, use_cnn=False, cnn_kernels='3', cnn_out_channels=50, use_crf=True, crf_loss_func='nll')

# data_args= DataTrainingArguments(data_dir='./nlpProjectNew/m_proj/BC5CDR-chem-IOB', labels='./labels.txt', train_split='train.txt', dev_split='dev.txt', test_split='test.txt', max_seq_length=256, overwrite_cache=False, metric_file_prefix='metrics_debug', metric_file_path='eval.tsv', use_da=False, weak_file=None, weak_wei_file=None, weak_dropo=False, weak_only=False, pred_file=None, save_pred_file=None, save_pred_rule=None, do_profile=False, profile_file='dev', no_eval=False, max_weight=1.0)

In [1]:
"""Training/validation/profile's main entry."""
print("reached run ner")
import os
path = "./stage_3_fineTuning/"
print("path=",path)
try:
    os.mkdir(path)
except OSError:
    print("Creation of the directory %s failed" % path)
else:
    print("Successfully created the directory %s " % path)

reached run ner
path= ./stage_3_fineTuning/
Successfully created the directory ./stage_3_fineTuning/ 


In [ ]:
# nerProcessor = DataProcessor('./nlpProjectNew/m_proj/BC5CDR-chem-IOB')
features_dim = {}
# print("nerProcessor=",nerProcessor)
print("features_dim=",features_dim)
label_list = ['O', 'B-Chemical', 'I-Chemical']
print("label_list=",label_list)
label_map = {'O': 0, 'B-Chemical': 1, 'I-Chemical': 2}
print("label_map=",label_map)
inversed_label_map = {0: 'O', 1: 'B-Chemical', 2: 'I-Chemical'}
print("inversed_label_map=",inversed_label_map)

nerProcessor= <preprocess.DataProcessor object at 0x791488dff640>
features_dim= {}
label_list= ['O', 'B-Chemical', 'I-Chemical']
label_map= {'O': 0, 'B-Chemical': 1, 'I-Chemical': 2}
inversed_label_map= {0: 'O', 1: 'B-Chemical', 2: 'I-Chemical'}


In [ ]:
train_examples = nerProcessor.get_examples('./nlpProjectNew/m_proj/BC5CDR-chem-IOB/train.txt')
train_examples_wei = None
print("train_examples =",len(train_examples))



==== Reading Data ====


100%|██████████| 4560/4560 [00:00<00:00, 23333.53it/s]


==== Create Examples ====


4560it [00:00, 425533.45it/s]

In [ ]:
example = train_examples[0]
print(example.features)
print(example.guid)
print(example.labels)
print(example.words)

train_examples = 4560


[]
0
['B-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['Selegiline', '-', 'induced', 'postural', 'hypotension', 'in', 'Parkinson', "'", 's', 'disease', ':', 'a', 'longitudinal', 'study', 'on', 'the', 'effects', 'of', 'drug', 'withdrawal', '.']


In [ ]:
weak_examples = nerProcessor.get_examples('/content/drive/MyDrive/nlpProjectNew/chem_weak.txt')

==== Reading Data ====


100%|██████████| 126/126 [00:00<00:00, 9425.07it/s]


==== Create Examples ====


(raylet) [2023-11-05 08:38:27,587 E 5866 5903] (raylet) agent_manager.cc:70: The raylet exited immediately because one Ray agent failed, agent_name = dashboard_agent/424238335.
(raylet) The raylet fate shares with the agent. This can happen because
(raylet) - The version of `grpcio` doesn't follow Ray's requirement. Agent can segfault with the incorrect `grpcio` version. Check the grpcio version `pip freeze | grep grpcio`.
(raylet) - The agent failed to start because of unexpected error or port conflict. Read the log `cat /tmp/ray/session_latest/logs/{dashboard_agent|runtime_env_agent}.log`. You can find the log file structure here https://docs.ray.io/en/master/ray-observability/ray-logging.html#logging-directory-structure.
(raylet) - The agent is killed by the OS (e.g., out of memory).
126it [00:00, 247069.80it/s]


In [ ]:
if train_examples_wei is None:
    train_examples_wei = [1] * len(train_examples)
train_examples_wei = [min(1.0, w) for w in train_examples_wei]
dev_examples = nerProcessor.get_examples('/content/nlpProjectNew/m_proj/BC5CDR-chem-IOB/dev.txt')
test_examples = nerProcessor.get_examples('/content/nlpProjectNew/m_proj/BC5CDR-chem-IOB/test.txt')
print("dev_examples  =",len(dev_examples), dev_examples[0])
print("test_examples  =",len(test_examples), test_examples[0])

set_seed(1)
num_labels = len(label_list)
print("num_labels=",num_labels)
print("label_list=",label_list)

==== Reading Data ====


100%|██████████| 4581/4581 [00:00<00:00, 40693.28it/s]


==== Create Examples ====


4581it [00:00, 379710.42it/s]


==== Reading Data ====


100%|██████████| 4797/4797 [00:00<00:00, 42522.51it/s]


==== Create Examples ====


4797it [00:00, 383831.75it/s]


dev_examples  = 4581 <preprocess.InputNERExample object at 0x79137d941d20>
test_examples  = 4797 <preprocess.InputNERExample object at 0x79137cf76aa0>
num_labels= 3
label_list= ['O', 'B-Chemical', 'I-Chemical']


In [ ]:
# config = AutoConfig.from_pretrained('dmis-lab/biobert-v1.1',num_labels=num_labels)
# # print("config=",config)

Downloading:   0%|          | 0.00/462 [00:00<?, ?B/s]

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained('dmis-lab/biobert-v1.1')
# print("tokenizer=",tokenizer)

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# load_name_or_path = 'dmis-lab/biobert-v1.1'

# model = NERModel.from_pretrained(
#     load_name_or_path,
#     from_tf=bool(".ckpt" in load_name_or_path),
#     config=config,
#     cache_dir=None,
# )
# print("model=",model)
print("=" * 80)
print("pad_token_label_id : ", pad_token_label_id)
label_map['[CLS]'] = pad_token_label_id
label_map['[SEP]'] = pad_token_label_id
label_map['X'] = pad_token_label_id
label_map['X'] = 0

# label_map['O'] = pad_token_label_id
# predicted_label_ids = torch.argmax(logits, dim=2).squeeze().tolist()

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

Some weights of NERAddon were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['crf_layer.start_transitions', 'crf_layer._constraint_mask', 'classifier.weight', 'crf_layer.transitions', 'classifier.bias', 'crf_layer.end_transitions']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


pad_token_label_id :  -100


In [ ]:
label_map


{'O': 0,
 'B-Chemical': 1,
 'I-Chemical': 2,
 '[CLS]': -100,
 '[SEP]': -100,
 'X': 0}

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
train_examples[0]

In [ ]:
weak_dataset = NerDataset(weak_examples, tokenizer, label_map, 256)


In [ ]:
train_dataset = NerDataset(train_examples, tokenizer, label_map, 256, train_examples_wei)
dev_dataset = NerDataset(dev_examples, tokenizer, label_map, 256)
test_dataset = NerDataset(test_examples, tokenizer, label_map, 256)


In [ ]:
example = train_examples[6]
print(example.features)
print(example.guid)
print(example.labels)
print(example.words)

print("="*80)


example_train = train_dataset[6]
print("input_ids:", example_train.input_ids)
print("attention_mask:", example_train.attention_mask)
print("token_type_ids:", example_train.token_type_ids)
print("label_ids:", example_train.label_ids)
print("features:", example_train.features)
print("predict_mask:", example_train.predict_mask)
print("weight:", example_train.weight)

[]
6
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['RESULTS', ':', 'Head', '-', 'up', 'tilt', 'caused', 'systolic', 'orthostatic', 'hypotension', 'which', 'was', 'marked', 'in', 'six', 'of', '20', 'PD', 'patients', 'on', 'selegiline', ',', 'one', 'of', 'whom', 'lost', 'consciousness', 'with', 'unrecordable', 'blood', 'pressures', '.']
input_ids: [101, 3463, 1024, 2132, 1011, 2039, 17010, 3303, 25353, 16033, 10415, 2030, 2705, 28696, 4588, 1044, 22571, 12184, 3619, 3258, 2029, 2001, 4417, 1999, 2416, 1997, 2322, 22851, 5022, 2006, 7367, 23115, 18622, 2638, 1010, 2028, 1997, 3183, 2439, 8298, 2007, 4895, 2890, 27108, 20782, 2668, 15399, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
sentence = tokenizer.decode(example_train.input_ids, skip_special_tokens=True)
print(len(sentence.split()), len(example_train.input_ids))
print(sentence)
print(example_train.input_ids)
print(' '.join(example.words))

30 256
results : head - up tilt caused systolic orthostatic hypotension which was marked in six of 20 pd patients on selegiline, one of whom lost consciousness with unrecordable blood pressures.
[101, 3463, 1024, 2132, 1011, 2039, 17010, 3303, 25353, 16033, 10415, 2030, 2705, 28696, 4588, 1044, 22571, 12184, 3619, 3258, 2029, 2001, 4417, 1999, 2416, 1997, 2322, 22851, 5022, 2006, 7367, 23115, 18622, 2638, 1010, 2028, 1997, 3183, 2439, 8298, 2007, 4895, 2890, 27108, 20782, 2668, 15399, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
lst1 = [-100, 0, -100, -100, -100, -100, 0, 0, 0, 0, 0, 0, 0, -100, -100, -100, 0, -100, -100, -100, 0, -100, -100, -100, -100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -100, -100, -100, 0, 0, 0, 0, 0, 0, 0, 0]
count_of_zeros = lst1.count(0)
count_of_zeros



28

In [ ]:
import torch
from transformers import BertTokenizer, BertForTokenClassification
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import trange


train_dataset = NerDataset(train_examples, tokenizer, label_map, 256, train_examples_wei)
dev_dataset = NerDataset(dev_examples, tokenizer, label_map, 256)
test_dataset = NerDataset(test_examples, tokenizer, label_map, 256)

# label_list = train_dataset.token_type_ids
# label_map = {label: i for i, label in enumerate(label_list)}
num_labels = 4

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

# Define other training parameters.
max_seq_length = 128  # Adjust as needed
batch_size = 32
learning_rate = 3e-5
num_epochs = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [ ]:
!pip install pytorch-crf -q


In [ ]:
import torch
from torch import nn
from transformers import BertModel, BertTokenizer
from torchcrf import CRF


In [ ]:
batch_size= 8
weak_loader = DataLoader(
    dataset=weak_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=weak_dataset.dynamic_collator(tokenizer.pad_token_id),
)
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=train_dataset.dynamic_collator(tokenizer.pad_token_id),
)
optimizer = AdamW(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
example_weak = weak_dataset[6]
print("input_ids:", example_weak.input_ids)
print("attention_mask:", example_weak.attention_mask)
print("token_type_ids:", example_weak.token_type_ids)
print("label_ids:", example_weak.label_ids)
print("features:", example_weak.features)
print("predict_mask:", example_weak.predict_mask)
print("weight:", example_weak.weight)

label_map

input_ids: [101, 1996, 3466, 1997, 14963, 1998, 1997, 6541, 1011, 1998, 8247, 1011, 4748, 7389, 2121, 12863, 10851, 6074, 2006, 12649, 6693, 1998, 2006, 16935, 1997, 3590, 8197, 2046, 12649, 1999, 9350, 6638, 4442, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

{'O': 0,
 'B-Chemical': 1,
 'I-Chemical': 2,
 '[CLS]': -100,
 '[SEP]': -100,
 'X': 0}

In [ ]:
# example_train = train_dataset[6]
# print("input_ids:", example_train.input_ids)
# print("attention_mask:", example_train.attention_mask)
# print("token_type_ids:", example_train.token_type_ids)
# print("label_ids:", example_train.label_ids)
# print("features:", example_train.features)
# print("predict_mask:", example_train.predict_mask)
# print("weight:", example_train.weight)

print(example_train.input_ids, type(example_train.input_ids), device)
len(train_loader)

[101, 3463, 1024, 2132, 1011, 2039, 17010, 3303, 25353, 16033, 10415, 2030, 2705, 28696, 4588, 1044, 22571, 12184, 3619, 3258, 2029, 2001, 4417, 1999, 2416, 1997, 2322, 22851, 5022, 2006, 7367, 23115, 18622, 2638, 1010, 2028, 1997, 3183, 2439, 8298, 2007, 4895, 2890, 27108, 20782, 2668, 15399, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] <class 'list'> cpu


570

In [ ]:
num_labels = 6

import torch
import torch.nn as nn
from transformers import BertForTokenClassification, BertTokenizer
import torchcrf

# Load pretrained BERT model
bert_model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [ ]:
# crf = torchcrf.CRF(num_labels)

# class BertCRFModel(nn.Module):
#     def __init__(self, bert_model, crf):
#         super(BertCRFModel, self).__init__()
#         self.bert = bert_model
#         self.crf = crf

#     def forward(self, input_ids, attention_mask, token_type_ids, labels):
#         outputs = self.bert(
#             input_ids=input_ids,
#             attention_mask=attention_mask,
#             token_type_ids=token_type_ids,
#             labels=labels
#         )

#         logits = outputs.logits

#         if labels is not None:
#             # Calculate CRF loss during training
#             crf_loss = -self.crf(logits, labels)
#             return crf_loss
#         else:
#             # Decode the most likely sequence of labels during inference
#             decoded_labels = self.crf.decode(logits)
#             return decoded_labels

# # Instantiate the combined BERT-CRF model
# model = BertCRFModel(bert_model, crf)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)


In [ ]:
label_map

{'O': 0,
 'B-Chemical': 1,
 'I-Chemical': 2,
 '[CLS]': -100,
 '[SEP]': -100,
 'X': 0}

In [ ]:
import torch
import torch.nn.functional as F

# Create a sample list of size n * 3
n = 5
values = torch.randn(n, 3)

# Apply softmax along the last dimension (dimension 1)
softmaxed_values = F.softmax(values, dim=1)

# Find the index of the maximum value in each row
max_indices = torch.argmax(softmaxed_values, dim=1)

# Print the original values, softmaxed values, and max indices
print("Original values:")
print(values)
print("Softmaxed values:")
print(softmaxed_values)
print("Max indices:")
print(max_indices)


Original values:
tensor([[-0.4711,  0.6481, -0.0840],
        [ 0.7841,  0.9581, -1.6575],
        [ 0.2536, -0.9021,  0.0220],
        [-0.2676, -0.2227,  0.1653],
        [-0.7368,  1.5863,  1.5113]])
Softmaxed values:
tensor([[0.1807, 0.5533, 0.2661],
        [0.4392, 0.5226, 0.0382],
        [0.4744, 0.1494, 0.3763],
        [0.2787, 0.2915, 0.4297],
        [0.0484, 0.4937, 0.4580]])
Max indices:
tensor([1, 1, 0, 2, 1])


In [ ]:
# inversed_label_map= {0: 'O', 1: 'B-Chemical', 2: 'I-Chemical', -100 : '[CLS]'}
inversed_label_map= {0: 'O', 1: 'B-Chemical', 2: 'I-Chemical'}


In [ ]:
from crfutils import ConditionalRandomField, allowed_transitions


In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel
from torchcrf import CRF
class BertWithCRF(nn.Module):
    def __init__(self, num_labels):
        super(BertWithCRF, self).__init__()
        self.bert = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)
        self.hidden_size = self.bert.config.hidden_size
        self.constraints = allowed_transitions("BIO", inversed_label_map)
        self.crf_layer = ConditionalRandomField(config.num_labels, self.constraints)

    def forward(self, input_ids, attention_mask, labels):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        # print("outputs.keys : ", outputs.keys())
        # print("output : ", outputs['logits'])
        # sequence_output = outputs.last_hidden_state
        # emissions = sequence_output
        # softmaxed_values = F.softmax(outputs['logits'], dim=1)

        # # Find the index of the maximum value in each row
        # max_indices = torch.argmax(softmaxed_values, dim=1)
        mask = attention_mask.byte()
        print("outputs['logits']", outputs['logits'].shape)
        print(outputs['logits'])
        print("labels", labels.shape)
        print(labels)
        print("mask", mask.shape)
        print(mask)

        loss = -self.crf_layer(outputs['logits'], labels, mask=mask)
        # loss = -self.crf_layer(max_indices, labels, mask=mask)
        return loss


# Define your CRF model with 'BIO' labels
num_labels = 3  # 3 labels: 'B', 'I', 'O'
model = BertWithCRF(num_labels)
model = model.to(device)
# Training and Inference remain the same as in the previous response

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [ ]:
model

BertWithCRF(
  (bert): BertForTokenClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
   

In [ ]:

num_epochs = 5

# Training loop
for epoch in trange(num_epochs, desc="Epoch"):
    total_loss = 0
    model.train()
    for step, batch in (enumerate(train_loader)):
        # Extract individual tensors from the batch dictionary
        # print("np.array of input_ids", batch['input_ids'])
        # print(" of attention_mask", batch['attention_mask'])
        # print(" of token_type_ids", batch['token_type_ids'])
        # print(" of label_ids", batch['labels'])

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        label_ids = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=label_ids)
        loss = outputs
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

# Save the trained model if needed
# model.save_pretrained('path_to_save_model')

Epoch:   0%|          | 0/5 [00:02<?, ?it/s]

outputs['logits'] torch.Size([8, 56, 3])
tensor([[[-1.5641e-01, -2.2902e-01, -3.1508e-01],
         [-2.2763e-01, -7.7703e-02,  2.4057e-02],
         [-5.0634e-01, -4.7088e-02, -3.7076e-01],
         ...,
         [-4.0717e-01, -1.1141e-01, -2.0925e-01],
         [-3.0280e-01, -9.4391e-02, -1.5730e-01],
         [-1.1622e-01, -2.1840e-01, -3.8096e-02]],

        [[-3.2807e-01,  8.4919e-02, -9.4999e-02],
         [ 2.0744e-02, -1.1462e-02, -8.0141e-01],
         [-6.3079e-02,  4.1864e-02, -8.3062e-02],
         ...,
         [-4.0014e-01,  1.1340e-03, -1.5122e-01],
         [-5.2029e-01, -1.3810e-01,  4.7466e-02],
         [-4.7905e-01, -1.5079e-01, -1.3700e-01]],

        [[-4.2299e-01, -2.4362e-02, -8.5683e-02],
         [-5.0953e-01,  5.5753e-01, -8.1012e-02],
         [-6.3165e-01,  3.8019e-01, -4.4206e-01],
         ...,
         [-2.6028e-01, -4.3942e-01,  1.5277e-01],
         [-9.8006e-02, -6.9399e-02, -3.5160e-01],
         [-3.5957e-01,  7.2835e-02, -2.2469e-01]],

        ...

IndexError: ignored

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel
from torchcrf import CRF

class BertWithCRF(nn.Module):
    def __init__(self, num_labels):
        super(BertWithCRF, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.hidden_size = self.bert.config.hidden_size
        self.crf = CRF(num_labels, batch_first=True)

    def forward(self, input_ids, attention_mask, labels):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        emissions = sequence_output
        emissions = self.apply_transition_constraints(emissions, labels)

        if labels is not None:
            mask = attention_mask.byte()
            loss = -self.crf(emissions, labels, mask=mask, reduction='mean')
            return loss
        else:
            return self.crf.decode(emissions, mask=attention_mask.byte())

    def apply_transition_constraints(self, emissions, labels):
        batch_size, seq_length, num_labels = emissions.size()
        transition_scores = torch.zeros(num_labels, num_labels).to(emissions.device)

        for i in range(batch_size):
            for j in range(seq_length):
                from_label = labels[i, j] if labels is not None else 0

                # Handle special tokens and out-of-range label indices
                if from_label not in inverse_label_map:
                    from_label = inverse_label_map["O"]  # Default to "O" label

                from_tag = inverse_label_map[from_label]
                from_entity = from_tag[2:] if from_tag != "O" else None

                if j == 0:
                    for to_label in range(num_labels):
                        to_tag = inverse_label_map[to_label]
                        if from_tag == "START":
                            transition_scores[from_label, to_label] = to_tag in ("O", "B")
                else:
                    for to_label in range(num_labels):
                        to_tag = inverse_label_map[to_label]
                        to_entity = to_tag[2:] if to_tag != "O" else None
                        if from_tag == "START":
                            transition_scores[from_label, to_label] = to_tag in ("O", "B")
                        elif to_tag == "END":
                            transition_scores[from_label, to_label] = from_tag in ("O", "B", "I")
                        else:
                            can_transition = (
                                to_tag in ("O", "B") or
                                (to_tag == "I" and from_tag in ("B", "I") and from_entity == to_entity)
                            )
                            transition_scores[from_label, to_label] = can_transition

        # Expand the transition scores to match the emissions size
        transition_scores = transition_scores.unsqueeze(0).unsqueeze(0).expand(batch_size, seq_length, -1, -1)

        # Add transition scores to emissions
        emissions = emissions + transition_scores

        return emissions


# Define your CRF model with 'BIO' labels
num_labels = 3  # 3 labels: 'B', 'I', 'O'
inverse_label_map = {0: "B", 1: "I", 2: "O"}  # Map label indices to labels
model = BertWithCRF(num_labels)
model = model.to(device)

# Training and Inference remain the same as in the previous responses


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
label_map

{0: 'B', 1: 'I', 2: 'O', '[CLS]': -100, '[SEP]': -100, 'X': 0}

In [ ]:
# model1 = torch.load("/content/nlpProjectNew/pytorch_model.bin")

In [2]:
import torch
from tqdm import trange

device = 'cuda'  # or 'cpu' if you encounter GPU memory issues

model1 = model.to(device)
num_epochs = 5
for epoch in trange(num_epochs, desc="Epoch"):
    total_loss = 0
    model1.train()
    for step, batch in enumerate(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        label_ids = batch['labels'].to(device)

        outputs = model1(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, labels=label_ids)
        loss = outputs[0]  # The first element of outputs is the loss

        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("LOSS : ", (total_loss / len(train_loader)))

# Save the trained model if needed
model1.save_pretrained('./finalOpModel.bin')

Epoch: 100%|██████████| 5/5 [06:58<00:00, 83.63s/it]LOSS :  0.0054214032562449574


In [ ]:
text = " ".join(train_examples[6].words)

tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(text)))

input_ids = tokenizer.convert_tokens_to_ids(tokens)
attention_mask = [1] * len(input_ids)
token_type_ids = [0] * len(input_ids)

input_ids = torch.tensor(input_ids).unsqueeze(0).to('cpu')  # Move input tensors to the CPU
attention_mask = torch.tensor(attention_mask).unsqueeze(0).to('cpu')
token_type_ids = torch.tensor(token_type_ids).unsqueeze(0).to('cpu')

with torch.no_grad():
    logits = model1(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids).logits


predicted_label_ids = torch.argmax(logits, dim=2).squeeze().tolist()

In [ ]:
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=test_dataset.dynamic_collator(tokenizer.pad_token_id),
)

In [ ]:
device = 'cpu'
final_active_logits = []
final_active_labels = []
for step, batch in (enumerate(test_loader)):
    # Extract individual tensors from the batch dictionary
    # print(np.array(input_ids).shape)
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    token_type_ids = batch['token_type_ids'].to(device)
    label_ids = batch['labels'].to(device)
    features = batch['features'].to(device)
    predict_mask = batch['predict_mask'].to(device)
    weights = batch['weights'].to(device)


    with torch.no_grad():
        logits = model1(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids).logits


    active_loss = attention_mask.view(-1) == 1
    active_logits = logits.view(-1, num_labels)
    active_labels = torch.where(
        active_loss,
        label_ids.view(-1),
        torch.tensor(loss_fn.ignore_index).type_as(label_ids)
    )
    active_logits = active_logits.cpu().detach().numpy()
    active_labels = active_labels.cpu().detach().numpy()
    # print(active_logits.shape, active_labels.shape)
    final_active_labels.extend(active_labels)
    final_active_logits.extend(active_logits)

    # break



In [ ]:
pred_logits = np.argmax(final_active_logits, axis=1)
print(pred_logits.shape)

(375778,)


In [ ]:
import numpy as np

# Define constants for metrics
TOKEN_ACCURACY = "TOKEN_ACCURACY"
SPAN_ACCURACY = "SPAN_ACCURACY"
MEAN_TOKEN_PRECISION = "MEAN_TOKEN_PRECISION"
MEAN_TOKEN_RECALL = "MEAN_TOKEN_RECALL"
MEAN_SPAN_PRECISION = "MEAN_SPAN_PRECISION"
MEAN_SPAN_RECALL = "MEAN_SPAN_RECALL"

def compute_metrices(pred_labels, true_labels):
    # Flatten the prediction and true label lists
    pred_labels_flat = np.array(pred_labels).flatten()
    true_labels_flat = np.array(true_labels).flatten()

    # Compute token accuracy
    token_accuracy = np.mean(pred_labels_flat == true_labels_flat)

    # Convert labels to strings for span-level metrics
    pred_labels_str = [str(label) for label in pred_labels_flat]
    true_labels_str = [str(label) for label in true_labels_flat]

    # Compute span accuracy
    span_accuracy = int(np.array_equal(pred_labels_str, true_labels_str))

    # Compute precision and recall
    precision, recall, _, _ = precision_recall_fscore_support(true_labels_str, pred_labels_str, average='micro')

    # Create a dictionary of metrics
    metrics = {
        TOKEN_ACCURACY: token_accuracy,
        SPAN_ACCURACY: span_accuracy,
        MEAN_TOKEN_PRECISION: precision,
        MEAN_TOKEN_RECALL: recall,
        MEAN_SPAN_PRECISION: precision,
        MEAN_SPAN_RECALL: recall,
    }

    return metrics


In [ ]:
# def compute_metrics(pred_label, out_label_list):
#     metrics = compute_accuracy_labels(out_label_list, pred_label, bio_format=True, ignore_labels=set(["category"]))
#     result = {
#         "TOKEN_ACCURACY": metrics[TOKEN_ACCURACY],
#         "SPAN_ACCURACY": metrics[SPAN_ACCURACY],
#         "MEAN_TOKEN_PRECISION": metrics[MEAN_TOKEN_PRECISION],
#         "MEAN_TOKEN_RECALL": metrics[MEAN_TOKEN_RECALL],
#         "MEAN_SPAN_PRECISION": metrics[MEAN_SPAN_PRECISION],
#         "MEAN_SPAN_RECALL": metrics[MEAN_SPAN_RECALL],
#     }
#     result["metrics"] = metrics
#     return result

In [ ]:
pred_logits.shape, type(pred_logits)

((375778,), numpy.ndarray)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

def get_results(text):
    # text = " ".join(test_examples[56].words)

    tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(text)))

    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    attention_mask = [1] * len(input_ids)
    token_type_ids = [0] * len(input_ids)

    input_ids = torch.tensor(input_ids).unsqueeze(0).to('cpu')  # Move input tensors to the CPU
    attention_mask = torch.tensor(attention_mask).unsqueeze(0).to('cpu')
    token_type_ids = torch.tensor(token_type_ids).unsqueeze(0).to('cpu')

    with torch.no_grad():
        logits = model1(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids).logits

    predicted_label_ids = torch.argmax(logits, dim=2).squeeze().tolist()


    tagged_words = []
    current_word = ""
    current_label = ""

    # Loop through tokens, subwords, and their corresponding predicted labels
    for token, predicted_label_id in zip(tokens, predicted_label_ids):
        label = list(label_map.keys())[list(label_map.values()).index(predicted_label_id)]

        if token.startswith("##"):
            current_word += token[2:]
        else:
            # If we have an existing entity, add it to the tagged_words list
            if current_word:
                tagged_words.append((current_word, current_label))
            current_word = token
            current_label = label

    # Add the last entity if any
    if current_word:
        tagged_words.append((current_word, current_label))

    # Print the tagged words and their labels
    words = []
    tags = []
    for word, label in tagged_words:
        words.append(word)
        tags.append(label)
    return words, tags


In [ ]:
with open('/content/nlpProjectNew/amazon-weak-ner-needle-main/bio_script/data/unlabeled_data/all_text.txt', 'r') as file:
    # Read the entire file contents
    file_contents = file.readlines()

# get_results(text)

In [ ]:
len(file_contents)
file_contents[0]

'Formate assay in body fluids: application in methanol poisoning.\n'

In [ ]:
sent, label = get_results(file_contents[0])
print(sent)
print(label)
print("="*80)

['[CLS]', 'formate', 'assay', 'in', 'body', 'fluids', ':', 'application', 'in', 'methanol', 'poisoning', '.', '[SEP]']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'O', 'O', 'O']


In [ ]:
sent, label = get_results(file_contents[2])
print(sent)
print(label)
print("="*80)

['[CLS]', 'effect', 'of', 'chloroquine', 'on', 'cultured', 'fibroblasts', ':', 'release', 'of', 'lysosomal', 'hydrolases', 'and', 'inhibition', 'of', 'their', 'uptake', '.', '[SEP]']
['O', 'O', 'O', 'B-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:
for i in range(30):
    print(file_contents[i])
    sent, label = get_results(file_contents[i])
    print(sent)
    print(label)
    print("="*80)

Formate assay in body fluids: application in methanol poisoning.

['[CLS]', 'formate', 'assay', 'in', 'body', 'fluids', ':', 'application', 'in', 'methanol', 'poisoning', '.', '[SEP]']
['O', 'B-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'O', 'O', 'O']
Delineation of the intimate details of the backbone conformation of pyridine nucleotide coenzymes in aqueous solution.

['[CLS]', 'delineation', 'of', 'the', 'intimate', 'details', 'of', 'the', 'backbone', 'conformation', 'of', 'pyridine', 'nucleotide', 'coenzymes', 'in', 'aqueous', 'solution', '.', '[SEP]']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Chemical', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Effect of chloroquine on cultured fibroblasts: release of lysosomal hydrolases and inhibition of their uptake.

['[CLS]', 'effect', 'of', 'chloroquine', 'on', 'cultured', 'fibroblasts', ':', 'release', 'of', 'lysosomal', 'hydrolases', 'and', 'inhibition', 'of', 'their', 'uptake', '.', '[SEP]']
['O', 'O', 'O', 'B-Ch

In [ ]:
device = 'cpu'
final_active_logits_noise = []
final_active_labels_noise = []
for step, batch in (enumerate(weak_loader)):
    # Extract individual tensors from the batch dictionary
    # print(np.array(input_ids).shape)
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    token_type_ids = batch['token_type_ids'].to(device)
    label_ids = batch['labels'].to(device)
    features = batch['features'].to(device)
    predict_mask = batch['predict_mask'].to(device)
    weights = batch['weights'].to(device)


    with torch.no_grad():
        logits = model1(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids).logits


    active_loss = attention_mask.view(-1) == 1
    active_logits = logits.view(-1, num_labels)
    active_labels = torch.where(
        active_loss,
        label_ids.view(-1),
        torch.tensor(loss_fn.ignore_index).type_as(label_ids)
    )
    active_logits = active_logits.cpu().detach().numpy()
    active_labels = active_labels.cpu().detach().numpy()
    # print(active_logits.shape, active_labels.shape)
    final_active_labels.extend(active_labels)
    final_active_logits_noise.extend(active_logits)


pred_logits_noise = np.argmax(final_active_logits_noise, axis=1)

(375778,)


In [ ]:
print(pred_logits_noise.shape)
pred_logits_noise[0]

(11482,)


0

In [ ]:
# train_loader = DataLoader(
#     dataset=train_dataset,
#     batch_size=batch_size,
#     shuffle=True,
#     collate_fn=train_dataset.dynamic_collator(tokenizer.pad_token_id),
# )

In [ ]:
weak_loader = DataLoader(
    dataset=weak_dataset,
    batch_size=1,
    shuffle=True,
    collate_fn=weak_dataset.dynamic_collator(tokenizer.pad_token_id),
)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
model1 = model1.to('cpu')
def get_results_noise(weak_loader):
    # text = " ".join(test_examples[56].words)

    # tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(text)))
    tagged_words = []
    real_label_id = []

    # input_ids = tokenizer.convert_tokens_to_ids(tokens)
    for i, batch in enumerate(weak_loader):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        token_type_ids = batch['token_type_ids']
        label_ids = batch['labels']
        features = batch['features']
        predict_mask = batch['predict_mask']
        weights = batch['weights']


        # input_ids = torch.tensor(input_ids).unsqueeze(0).to('cpu')  # Move input tensors to the CPU
        # attention_mask = torch.tensor(attention_mask).unsqueeze(0).to('cpu')
        # token_type_ids = torch.tensor(token_type_ids).unsqueeze(0).to('cpu')

        with torch.no_grad():
            logits = model1(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids).logits
        # print(np.array(logits).shape, np.array(input_ids).shape)
        predicted_label_ids = torch.argmax(logits, dim=2).squeeze().tolist()

        # print(np.array(predicted_label_ids).shape)
        # print(np.array(label_ids).shape)
        # print(np.array(tokens).shape)
        # break
        current_word = ""
        current_label = ""
        print(len(predicted_label_ids) , len(label_ids))
        if(len(predicted_label_ids) == len(label_ids)):
            print("Entered")
            for token, predicted_label_id, labe in zip(tokens, predicted_label_ids, label_ids):
                label = list(label_map.keys())[list(label_map.values()).index(predicted_label_id)]

                if token.startswith("##"):
                    current_word += token[2:]
                else:
                    if current_word:
                        tagged_words.append((current_word, current_label))
                    current_word = token
                    current_label = label
                    real_label_id.append(labe)

            if current_word:
                tagged_words.append((current_word, current_label))

        # print("="*80)
        # print(np.array(predicted_label_ids).shape)
        # print(np.array(label_ids).shape)
        # break
    words = []
    tags = []
    real_ids = []
    return_tags = []
    # final_labels_noise = waek_data_labels
    # for i in range(len(tags)):
    #     if(real_label_id[i] == 0 and tags[i] != 0):
    #         return_tags[i] = tags[i]

    for word, label in enumerate(tagged_words):
        words.append(word)
        if(real_label_id[i] == 0 and label[i] != 0):
            return_tags[i] = tags[i]
        return_tags.append(label_ids[i])
        # tags.append(label)
    return words, return_tags


In [ ]:
weak_data_sent, waek_data_labels= get_results_noise(weak_loader)


In [ ]:
# print(len(weak_data_sent), np.array(weak_data_sent[0:50]).shape)
# print(len(waek_data_labels), np.array(waek_data_labels[:50]).shape)
# print(len(real_label_id), np.array(real_label_id[:50]).shape)

0 (0,)
0 (0,)
0 (0,)


In [ ]:
import numpy as np

# Define constants for metrics
TOKEN_ACCURACY = "TOKEN_ACCURACY"
SPAN_ACCURACY = "SPAN_ACCURACY"
MEAN_TOKEN_PRECISION = "MEAN_TOKEN_PRECISION"
MEAN_TOKEN_RECALL = "MEAN_TOKEN_RECALL"
MEAN_SPAN_PRECISION = "MEAN_SPAN_PRECISION"
MEAN_SPAN_RECALL = "MEAN_SPAN_RECALL"

def compute_metrices1(pred_labels, true_labels):
    # Flatten the prediction and true label lists
    pred_labels_flat = np.array(pred_labels).flatten()
    true_labels_flat = np.array(true_labels).flatten()

    # Compute token accuracy
    true_positive, false_positive = 0, 0
    # token_accuracy = np.mean(pred_labels_flat == true_labels_flat)
    for i in range(min(len(pred_labels_flat), len(true_labels_flat))):
        if(true_labels_flat[i] == 1 or true_labels_flat[i] == 2):
            if(true_labels_flat[i] == pred_labels_flat[i]):
                true_positive += 1
            else:
                false_positive += 1

    token_accuracy = true_positive/(true_positive + false_positive)

    # # Convert labels to strings for span-level metrics
    # pred_labels_str = [str(label) for label in pred_labels_flat if label in [1, 2]]
    # true_labels_str = [str(label) for label in true_labels_flat if label in [1, 2]]
    pred_labels_str, true_labels_str = [], []
    for i in range(min(len(pred_labels_flat), len(true_labels_flat))):
        if(true_labels_flat[i] == 1 or true_labels_flat[i] == 2):
            pred_labels_str.append(pred_labels_flat[i])
            true_labels_str.append(true_labels_flat[i])



    # Compute span accuracy
    span_accuracy = int(np.array_equal(pred_labels_str, true_labels_str))

    # Compute precision and recall
    precision, recall, _, _ = precision_recall_fscore_support(true_labels_str, pred_labels_str, average='micro')

    # Create a dictionary of metrics
    metrics = {
        TOKEN_ACCURACY: token_accuracy,
        SPAN_ACCURACY: span_accuracy,
        MEAN_TOKEN_PRECISION: precision,
        MEAN_TOKEN_RECALL: recall,
        MEAN_SPAN_PRECISION: precision,
        MEAN_SPAN_RECALL: recall,
    }

    return metrics

In [ ]:
compute_metrices1(pred_logits, final_active_labels)

{'TOKEN_ACCURACY': 0.8514187936689006,
 'SPAN_ACCURACY': 0,
 'MEAN_TOKEN_PRECISION': 0.8514187936689006,
 'MEAN_TOKEN_RECALL': 0.8514187936689006,
 'MEAN_SPAN_PRECISION': 0.8514187936689006,
 'MEAN_SPAN_RECALL': 0.8514187936689006}

In [ ]:
text = " ".join(test_examples[56].words)

tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(text)))

input_ids = tokenizer.convert_tokens_to_ids(tokens)
attention_mask = [1] * len(input_ids)
token_type_ids = [0] * len(input_ids)

input_ids = torch.tensor(input_ids).unsqueeze(0).to('cpu')  # Move input tensors to the CPU
attention_mask = torch.tensor(attention_mask).unsqueeze(0).to('cpu')
token_type_ids = torch.tensor(token_type_ids).unsqueeze(0).to('cpu')

with torch.no_grad():
    logits = model1(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids).logits

predicted_label_ids = torch.argmax(logits, dim=2).squeeze().tolist()


tagged_words = []
current_word = ""
current_label = ""

# Loop through tokens, subwords, and their corresponding predicted labels
for token, predicted_label_id in zip(tokens, predicted_label_ids):
    label = list(label_map.keys())[list(label_map.values()).index(predicted_label_id)]

    if token.startswith("##"):
        current_word += token[2:]
    else:
        # If we have an existing entity, add it to the tagged_words list
        if current_word:
            tagged_words.append((current_word, current_label))
        current_word = token
        current_label = label

# Add the last entity if any
if current_word:
    tagged_words.append((current_word, current_label))

# Print the tagged words and their labels
for word, label in tagged_words:
    print(f"Word: {word}, Label: {label}")


Word: [CLS], Label: O
Word: the, Label: O
Word: catechol, Label: B-Chemical
Word: and, Label: O
Word: hydroquinone, Label: B-Chemical
Word: metabolites, Label: O
Word: ,, Label: O
Word: ncq436, Label: O
Word: and, Label: O
Word: ncq344, Label: O
Word: ,, Label: O
Word: induced, Label: O
Word: apoptosis, Label: O
Word: in, Label: O
Word: hl60, Label: O
Word: and, Label: O
Word: hbmp, Label: O
Word: cells, Label: O
Word: in, Label: O
Word: a, Label: O
Word: time, Label: O
Word: -, Label: O
Word: and, Label: O
Word: concentration, Label: O
Word: dependent, Label: O
Word: manner, Label: O
Word: ,, Label: O
Word: while, Label: O
Word: the, Label: O
Word: phenols, Label: O
Word: ,, Label: O
Word: ncr181, Label: O
Word: ,, Label: O
Word: fla873, Label: O
Word: ,, Label: O
Word: and, Label: O
Word: fla797, Label: O
Word: ,, Label: O
Word: and, Label: O
Word: the, Label: O
Word: derivatives, Label: O
Word: formed, Label: O
Word: by, Label: O
Word: oxidation, Label: O
Word: of, Label: O
Word: th